In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("chinook.db")

In [41]:
emp = pd.read_sql_query("select (FirstName||' '||LastName) as FullName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Phone, Email from employees", conn)
emp.head()

,FullName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Phone,Email
0,Andrew Adams,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,+1 (780) 428-9482,andrew@chinookcorp.com
1,Nancy Edwards,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,+1 (403) 262-3443,nancy@chinookcorp.com
2,Jane Peacock,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,+1 (403) 262-3443,jane@chinookcorp.com
3,Margaret Park,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,+1 (403) 263-4423,margaret@chinookcorp.com
4,Steve Johnson,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,1 (780) 836-9987,steve@chinookcorp.com


In [8]:
track = pd.read_sql_query("select * from tracks where TrackId = 4", conn)
track

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99


In [15]:
genre = pd.read_sql_query("select trk.Name as TrackNm, trk.Composer, trk.UnitPrice, gnr.Name as GenreNm, gnr.GenreId, trk.TrackId \
                   from tracks as trk, genres as gnr \
                   where trk.GenreId = gnr.GenreId and trk.TrackId = 4 and gnr.GenreId = 1", conn)
genre.head()

,TrackNm,Composer,UnitPrice,GenreNm,GenreId,TrackId
0,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",0.99,Rock,1,4


In [22]:
pd.read_sql_query("SELECT * FROM media_types", conn)

,MediaTypeId,Name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [38]:
ttl_sales = pd.read_sql_query("SELECT inv.BillingCountry as Country, gnr.Name as Genre, inv.InvoiceDate, SUM(inv.Total) as Total \
                   FROM invoices as inv, invoice_items as inv_itm, tracks as trk, genres as gnr, media_types as md_typ \
                   WHERE \
                   inv.InvoiceId = inv_itm.InvoiceId \
                   AND \
                   inv_itm.TrackId = trk.TrackId \
                   AND \
                   trk.GenreId = gnr.GenreId \
                   AND \
                   trk.MediaTypeId = md_typ.MediaTypeId \
                   AND \
                   md_typ.MediaTypeId = 3 \
                   GROUP BY Genre \
                   ORDER BY Total", conn, parse_dates='InvoiceDate')

ttl_sales['Month'] = ttl_sales["InvoiceDate"].dt.month_name()
ttl_sales

,Country,Genre,InvoiceDate,Total,Month
0,Chile,Science Fiction,2010-01-13,102.41,January
1,USA,Comedy,2012-09-27,112.30,September
2,USA,Sci Fi & Fantasy,2012-08-05,198.87,August
3,Czech Republic,Drama,2013-11-13,544.61,November
4,Sweden,TV Shows,2010-01-10,817.71,January


In [32]:
country_sales = pd.read_sql_query("SELECT inv.BillingCountry as Country, gnr.Name as Genre, md_typ.Name as Media_Nm, sum(inv.Total) as Total\
                   FROM invoices as inv, invoice_items as inv_itm, tracks as trk, genres as gnr, media_types as md_typ \
                   WHERE \
                   inv.InvoiceId = inv_itm.InvoiceId \
                   AND \
                   inv_itm.TrackId = trk.TrackId \
                   AND \
                   trk.GenreId = gnr.GenreId \
                   AND \
                   trk.MediaTypeId = md_typ.MediaTypeId \
                   AND \
                   inv.BillingCountry = 'Germany' \
                   GROUP BY Genre \
                   ORDER BY Total DESC", conn)
country_sales

,Country,Genre,Media_Nm,Total
0,Germany,Rock,Protected AAC audio file,577.17
1,Germany,Metal,MPEG audio file,189.09
2,Germany,Latin,MPEG audio file,163.35
3,Germany,Alternative & Punk,MPEG audio file,105.12
4,Germany,Blues,MPEG audio file,97.02
5,Germany,Soundtrack,MPEG audio file,59.40
6,Germany,TV Shows,Protected MPEG-4 video file,44.73
7,Germany,Heavy Metal,MPEG audio file,41.58
8,Germany,Science Fiction,Protected MPEG-4 video file,29.82
9,Germany,Jazz,MPEG audio file,27.72


In [46]:
ttl_sales_media = pd.read_sql_query("SELECT inv.BillingCountry as Country, gnr.Name as Genre, md_typ.Name, SUM(inv.Total) as Total \
                   FROM invoices as inv, invoice_items as inv_itm, tracks as trk, genres as gnr, media_types as md_typ \
                   WHERE \
                   inv.InvoiceId = inv_itm.InvoiceId \
                   AND \
                   inv_itm.TrackId = trk.TrackId \
                   AND \
                   trk.GenreId = gnr.GenreId \
                   AND \
                   trk.MediaTypeId = md_typ.MediaTypeId \
                   AND \
                   md_typ.MediaTypeId = 3 \
                   GROUP BY Genre \
                   ORDER BY Total", conn)
ttl_sales_media

,Country,Genre,Name,Total
0,Chile,Science Fiction,Protected MPEG-4 video file,102.41
1,USA,Comedy,Protected MPEG-4 video file,112.30
2,USA,Sci Fi & Fantasy,Protected MPEG-4 video file,198.87
3,Czech Republic,Drama,Protected MPEG-4 video file,544.61
4,Sweden,TV Shows,Protected MPEG-4 video file,817.71
